### Лекция 4. Классы продолжение

[Back to Basics: Designing Classes (part 1 of 2) - Klaus Iglberger - CppCon 2021](https://youtu.be/motLOioLJfg)  
[Back to Basics: Designing Classes (part 2 of 2) - Klaus Iglberger - CppCon 2021](https://youtu.be/O65lEiYkkbc)  
[Non standard layout guarantees](https://quuxplusone.github.io/blog/2022/03/04/non-standard-layout-guarantees/)

<br />

##### Спецификация вызова конструктора базового класса

Базовый класс может иметь несколько конструкторов, а наследник может выбирать, какой именно из конструкторов он хочет вызвать. Это определяется в списке инициализации конструктора

```c++
class Animal
{
public:
    Animal();
    Animal(const std::string& a_name);
    Animal(const std::string& a_name, unsigned a_age);

private:
    std::string name;
    unsigned age;
};

class Turtle : public Animal
{
public:
    // неявно вызывает Animal()
    Turtle() {}    
    // эквивалентно записи:
    // Turtle() : Animal() {}
    
    // явный вызов конкретного конструктора Animal
    Turtle(const std::string& name)
        : Animal(name)  // сначала вызываем конструктор базового класса
        , color(White)  // затем конструкторы членов в порядке их определения
    {
    }
    
private:
    Color color = Black;  // значение для инициализации по умолчанию
};
```

<br />

##### Виртуальные функции

Задача, которую решает механизм виртуальных функций: класс-наследник может __уточнять__ поведение базового класса

Примеры:

```c++
class Animal
{
public:
    virtual void cry() {}
};

class Dog : public Animal
{
public:
    void cry() {
        std::cout << "augh!" << std::endl;
    }
};

class Cat : public Animal
{
public:
    void cry() {
        std::cout << "meow!" << std::endl;
    }
};
```

__Вопрос знатокам__:

<details>
<summary>Какая ошибка здесь допущена?</summary>
<p>
    
Нет виртуального деструктора у `Animal`. Объяснение на семинаре.
    
</p>    
</details>

<details>
<summary>Почему это не страшно? (но поправить ошибку всё равно желательно)</summary>
<p>
    
Деструкторы Cat && Dog ничего не делает.
    
</p>    
</details>

```c++
void make_it_cry(Animal& animal)
{
    animal.cry();
    animal.cry();
    animal.cry();
}

Animal animal;
make_it_cry(animal);

Dog dog;
make_it_cry(dog);
```

А что здесь?

```c++
void make_it_cry_more(Animal animal)
{
    for (int i = 0; i < 100; ++i)
        animal.cry();
}

Animal animal;
make_it_cry_more(animal);

Dog dog;
make_it_cry_more(dog);
```

<br />

##### Чисто виртуальные функции

Метод `cry()` не имеет смысла для класса `Animal`, т.к. не понятно, какая реализация у него должна быть. Можно указать, что у виртуальной функции `Animal::cry()` нет реализации. Такие функции называются **чисто виртуальными**.

```c++
class Animal
{
public:
    virtual void cry() = 0;
};

// далее аналогично ...
class Dog : public Animal { ... };
class Cat : public Animal { ... };
```

__Вопрос:__

<details>
<summary>Что ещё принципиально поменялось для класса Animal?</summary>
<p>

Объекты типа `Animal` нельзя создавать.

</p>
</details>

Ещё пример:

```c++
class BridgeBuilder
{
public:
    virtual Bridge make() = 0;    
};

class StoneBridgeBuilder : public BridgeBuilder
{
public:
    Bridge make() { return Bridge("stone"); }    
};

class WoodBridgeBuilder : public BridgeBuilder
{
public:
    Bridge make() { return Bridge("wood"); }    
};

Landscape make_landscape(BridgeBuilder& bridge_builder)
{
    const auto bridge1 = bridge_builder.make();
    const auto bridge2 = bridge_builder.make();
    ...;
}
```

__Вопрос знатокам:__

<details>
<summary>Что-нибудь напоминает?</summary>
<p>

Abstract Factory pattern.

</p>
</details>

С философской точки зрения ООП чисто виртуальные функции лучше отражают ООП-модель программы. Но и обычные виртуальные функции применимы.

Пример:

```c++
// класс для работы с сетью
class Network
{
public:
    virtual HttpResponse get(const std::string& url) { /*...*/ }
};

// класс для тестирования медленной сети
class SlowNetwork : public Network
{
public:
    virtual HttpResponse get(const std::string& url) {
        // спим
        std::chrono::sleep_for(100_ms);
        
        // вызов базового класса
        return Network::get(url);
    }
};
```

<br />

##### virtual / override / final

* `virtual` - сделать метод виртуальным, чтобы наследники могли его переопределять
* `override` - указание компилятору убедиться, что метод виртуальный, если нет - ошибка компиляции
* `final` - запретить наследникам переопределять виртуальный метод

Зачем нужен `override`:

Иерархия сегодня:

```c++
class Animal
{
public:
    virtual void cry() {}
};

class Dog : public Animal
{
public:
    void cry() { std::cout << "augh!"; }
};
```

Иерархия через пару месяцев:

```c++
class Animal
{
public:
    virtual void cry(bool loud) {}
};

class Dog : public Animal
{
public:
    void cry() { std::cout << "augh!"; } // OOOPS, it comiles but is not working as expected anymore
};
```

Как надо было делать иерархию "сегодня":

```c++
class Animal
{
public:
    virtual void cry(bool cloud) {}    
};

class Dog : public Animal
{
public:
    void cry() override { std::cout << "augh!"; }
};
```

Тогда ошибку поймал бы компилятор, а не пользователь.

<br />

##### vftable

Один из способов реализации механизмов виртуальных функций - компилятор может вписать в класс дополнительное поле - указатель на таблицу виртуальных функций:

```c++
class Animal
{
    std::uint64_t age;
public:
    virtual void cry();
    virtual void jump();
};

class Dog : public Animal
{
    std::uint64_t color;
public:
    void cry() override;
    void jump() override;
};
```

![](vftable.jpg)

Вопросы:

<details>
<summary>sizeof(Animal)</summary>
16
</details>
    
<details>
<summary>sizeof(Dog)</summary>
24
</details>

<br />

##### Множественное наследование, ромбовидное наследование

Обратить внимание на порядок конструирования, вызова деструкторов и способа разрешения конфликтов по именам

```c++
class Animal {};

class JumpingCreature
{
public:
    void jump();
};

class CryingCreature
{
public:
    void cry();    
};


class Cat : public Animal
          , public JumpingCreature
          , public CryingCreature
{
public:
    void play()
    {
        jump();
        cry();
        jump();
        
        // явное указание метода,
        //
        // потребуется, если метод jump()
        // есть и у JumpingCreature, и у Animal
        JumpingCreature::jump();
        CryingCreature::cry();
    }
};
```

<img src="classes_diagram_cat.png" height=500px width=500px />

<br />

Граф наследования ацикличен и однонаправлен... но ничто не мешает сделать в нём ромб. Такая конструкция выглядит необычной, но, тем не менее, используется даже в стандартной библиотеке:

![](std-basic_iostream-inheritance.png)

https://en.cppreference.com/w/cpp/io/basic_iostream

<br />

##### Layout

Рассмотрим, как устроено расположение класса в памяти. Пойдём от простого к сложному.

```c++
struct Point
{
    float x;  // size = 4, alignment = 4
    float y;  // size = 4, alignment = 4
};  // size = 8, alignment = 4
```

![](layout_point.jpg)

```c++
class A
{
    float x;          // size = 4, alignment = 4
    std::uint64_t y;  // size = 8, alignment = 8  
};  // size = ???, alignment = ???
```

![](layout_padding.jpg)

Более подробно про alignment:
https://en.cppreference.com/w/c/language/object

```c++
class A
{
    float x;          // size = 4, alignment = 4
    std::uint64_t y;  // size = 8, alignment = 8
    float z;          // size = 4, alignment = 4
};  // size = ???, alignment = ???
```

![](layout_padding_2.jpg)

```c++
class A
{
    std::uint64_t y;  // size = 8, alignment = 8
    float x;          // size = 4, alignment = 4
    float z;          // size = 4, alignment = 4
};  // size = ???, alignment = ???
```

![](layout_padding_3.jpg)

<br />

Рассмотрим layout класса в памяти в случае наследования:

```c++
struct P
{
    float x;  // size = 4, alignment = 4
    float y;  // size = 4, alignment = 4
};  // size = 8, alignment = 4

struct WP : P
{
    float w;  // size = 4, alignment = 4
};  // size = ???, alignment = ???
```

![](layout_inheritance.jpg)

Рассмотрим layout класса в памяти в случае множественного наследования:

```c++
class CBase1 { ... };
class CBase2 { ... };

class CDerived : public CBase1
               , public CBase2
{ ... };
```

![](layout_multiple_inheritance.jpg)

<br />

Как быть с виртуальными методами и функциями:

```c++
class Base
{
public:
    virtual void say_hello();
    virtual void say_goodbye();
    
private:
    ...
};

class Dervied : public Base
{
public:
    void say_hello() override;
    void say_goodbye() override;
    
private:
    ...
};
```

![](layout_vtable_3.jpg)

Показать как ведёт себя преобразование указателей для такого лаяута (и почему с reinterpert_cast лучше не переборщить).

Открыть на godbolt.org пример и показать как происходит сравнение указателей, закомментировать виртуальные функции - посмотреть что происходит.

```c++
struct Base1
{
    // virtual void say_hello() = 0;
    float x;
};

struct Base2
{
    float x;
};

struct Derived : public Base1
               , public Base2
{
    // virtual void say_hello() = 0;
    float y;
};

bool equal1(Base1* lhs, Derived *rhs)
{
    return lhs == rhs;
}

bool equal2(Base2* lhs, Derived *rhs)
{
    return lhs == rhs;
}
```

__Вопрос на понимание:__ каков layout класса при ромбовидном наследовании? (лучше нарисовать)

__Вопрос для продвинутых:__ как его оптимизировать?

<br />

##### Состояние класса, const + mutable

Что считать состоянием класса?
* значения всех его полей
* логическое состояние класса (набор элементов в векторе, а не детали типа `capacity`)

С точки зрения компилятора - набор всех его полей, не помеченных словом `mutable`

```c++
class Animal
{
private:
    std::string name;
    unsigned age;
    
public:
    // аттрибут const означает, что метод не будет напрямую
    // менять поля класса и вызывать не-const методы (и компилятор это проверит!)
    //
    // внутри const-методов поля имеют тип:
    // const std::string name;
    // const unsigned age;
    bool is_too_young() const
    {
        return age <= 3;
    }
    
    // метод не меняет полей класса, но программист
    // забыл поставить const
    void say_hello()
    {
        std::cout << "hello";
    }
    
    // метод не меняет полей класса
    void say_hello_twice() const
    {
        say_hello();  // COMPILE-TIME ERROR
        say_hello();
        
        name = "unk";  // COMPILE-TIME ERROR
    }
    
    // отсутствие const - метод МОЖЕТ менять поля
    void happy_birthday()
    {
        age += 1; // OK
    }   
};
```

**Зачем нужен `mutable`?**

Рассмотрим учебный пример - отображение, которое хранит ответ на последний запрос как кеширование.

```c++
class Resources
{
public:
    void add(int id, const std::string& resource);
    
    const std::string& get(int id) const;
    
private:
    // хранилище ресурсов
    std::map<int, std::string> id_to_resource;
    
    // закешированный последний запрошенный ресурс
    mutable int last_id = -1;
    mutable std::string last_resource;
};

void Resources::add(const int id, const std::string& resource)
{
    id_to_resource[id] = resource;
}

const std::string& Resources::get(const int id) const
{
    if (id == last_id)
        return last_resource;
    
    auto it = id_to_resource.find(id);  // |find| - const-метод
    
    last_resource = it->second;  // OK, т.к. mutable
    last_id = id;                // OK, т.к. mutable
    
    return last_resource;
}
```

Примечание: это ученический пример кеширования для демонстрации понятий `mutable` и состояния объекта, так кеширование реализовывать не следует.

<br />

##### Операторы

https://en.cppreference.com/w/cpp/language/operators

В виде свободных функций:

```c++
struct Vec
{
    float x;
    float y;
};

// v1 + v2
Vec operator+ (const Vec& l, const Vec& r)
{
    return {l.x + r.x, l.y + r.y};
}

// v1 * v2
float operator* (const Vec& l, const Vec& r)
{
    return l.x * r.x + l.y * r.y;
}

// v * a
Vec operator* (const Vec& v, const float a)
{
    return {v.x * a, v.y * a};
}

// a * v
Vec operator* (const float a, const Vec& v)
{
    return v * a;
}

// v / a
Vec operator/ (const Vec& v, const float a)
{
    return v * (1. / a);
}

// v1 += v2;
Vec& operator+= (Vec& l, const Vec& r)
{
    l.x += r.x;
    l.y += r.y;
    return l;
}
    
// v1 -= v2;
Vec& operator-= (Vec& l, const Vec& r)
{
    l.x -= r.x;
    l.y -= r.y;
    return l;
}

// v1 *= a
Vec& operator*= (Vec& l, const float a)
{
    v1.x *= a;
    v1.y *= a;
    return v1;
}

// v1 /= a
Vec& operator*= (Vec& l, const float a)
{
    l.x /= a;
    l.y /= a;
    return l;
}

// std::cout << v
std::ostream& operator <<(std::ostream& os, const Vec& v)
{
    os << v.x << ',' << v.y;
    return os;
}

bool operator< (const Vec& l, const Vec& r)
{
    return l.x != r.x ? l.x < r.x : l.y < r.y;
}

bool operator==(const Vec& l, const Vec& r)
{
    return l.x == r.x && l.y == r.y;
}

bool operator> (const Vec& l, const Vec& r) { return r < l; }
bool operator<=(const Vec& l, const Vec& r) { return !(l > r); }
bool operator>=(const Vec& l, const Vec& r) { return !(l < r); }
bool operator!=(const Vec& l, const Vec& r) { return !(l == r); }
```

В виде методов и функций - друзей:

```c++
// демонстрационный класс строки;
// класс строки из стандартной библиотеки похитрее
class String
{
private:
    char* data;
    size_t len;

public:
    String();
    String(const char* s);
    String(const String& rhs);
    String(String&& rhs);
    String& operator = (const String& rhs);
    String& operator = (String&& rhs);
    ~String();
    
    //
    // operator[] может быть только в виде метода
    //
    // char c = s[i];
    // s[i] = 'a';
    //
    // const String s = ....;
    // s[2] = 'a';
    char& operator[] (int i)
    {
        return data[i];
    }
    
    const char& operator[] (std::size_t i) const
    {
        return data[i];
    }
    
    // s += s1;
    String& operator+= (const String& rhs)
    {
        // какая-то реализация ...
        return *this;
    }
    
    // s1 + s2
    String operator+ (const String& rhs) const
    {
        String s = *this;
        s += rhs;
        return s;
    }

    // s1 *= n - повторить строку n раз
    String& operator*= (std::size_t n)
    {
        // какая-то реализация ...
        return *this;
    }    
    // Замечание: через операторы в виде методов сделать n * s1 нельзя
    
    // s1 * n
    String operator* (std::size_t n) const
    {
        String s = *this;
        s *= n;
        return s;
    }
    
    // std::cout << s
    friend std::ostream& operator<<(std::ostream& os, const String& v)
    {
        return os << v.data;
    }

    friend bool operator< (const String& l, const String& r)
    {
        return /*...*/;
    }

    friend bool operator==(const String& l, const String& r)
    {
        return /*...*/;
    }

    friend bool operator> (const Vec& l, const Vec& r) { return r < l; }
    friend bool operator<=(const Vec& l, const Vec& r) { return !(lhs > rhs); }
    friend bool operator>=(const Vec& l, const Vec& r) { return !(lhs < rhs); }
    friend bool operator!=(const Vec& l, const Vec& r) { return !(lhs == rhs); }
};
```

**Замечание**: разница между определением оператора внутри класса и вне класса:
https://www.quora.com/What-is-the-difference-between-defining-an-overloaded-operator-outside-the-class-and-defining-it-inside-the-class/answer/Sergey-Zubkov-1

<br />

##### static-поля класса

Класс может иметь `static`-поля и `static`-методы:

`static`-поля можно рассматривать как глобальные константы / переменные:

```c++

// Animal.h

class Animal
{
public:
    Animal(const std::string& a_name, unsigned a_age);
    ...
    
    // declaration
    static const std::string name_of_unknown;
};

// Animal.cpp

// definition
const std::string Animal::name_of_unknown = "UNK";

// main.cpp

// usage:
void func()
{
    std::cout << "name of uknown animal is: " << Animal::name_of_unknown << std::endl;
    
    Animal animal(Animal::name_of_unknown, 12);
}
```

На `static`-поля распространяются модификаторы области доступа `private`/`protected`/`public`:
* В случае `public` `static`-поле ведёт себя как обычная глобальная константа
* В случае `private` только методы класса имеют доступ до этой константы

__Вопрос__: 

<details>
<summary>Зачем могут быть полезны `static`-поля класса</summary>
<p>
   
* Классоспецифичные константы.
* Классоспецифичные глобальные переменные (по возможности избегать).
   
</p>
</details>


<details>
<summary>Ещё примеры?</summary>
<p>
   
* Класс `DateTime`. Можно в `public` вынести константы minute, second, hour ...
* Ещё?
   
</p>
</details>

<br />

##### static-методы класса

`static`-метод класса можно рассматривать как свободную friend-функцию:

```c++
class Animal
{
    int age = 0;

public:
    static void grow_up(int count, Animal& an)
    {
        an.age += count;
    }

    int get_age() const { return age; }
};

int main()
{
    Animal an;
    Animal::grow_up(4, an);  // TODO: обратить внимание на вызов
    std::cout << an.get_age() << std::endl;
    return 0;
}
```

`static`-метод - обычная friend-функция, находящаяся в "пространстве имён" класса.

**Для вызова static-метода _НЕ_ нужен объект класса**

Пример использования `static`-метода:

```c++
class House
{
    Floor floor;
    Roof roof;
    std::vector<Wall> walls;

public:
    House(Floor f, Roof r, std::vector<Wall> walls);
    
    static House make(const Config& config)
    {
        auto floor = Floor::make(config["floor"]);
        auto root = Roof::make(config["roof"]);
        auto walls = ...;        
        return House(floor, roof, walls);
    }    
};
```

__Вопрос__: что-нибудь напоминает?

<br />

##### Виртуальные функции

Какая функция будет вызвана. Нарисовать что будет происходить в памяти

```c++
struct A
{
	~A() {}
	void f() {}
	virtual void g() {}
	void h() {}
};

struct B : public A
{
	virtual ~B() {}
	virtual void f() {}
	void g() {}
	void h() {}
};

struct C : public B
{
	~C() {}
	void f() {}
	void g() {}
	virtual void h() {}
};

int main()
{
	{
		C c;
		c.f(); c.g(); C.h();

		B b = c;
		b.f(); b.g(); b.h();

		A& ar = c;
		ar.f(); ar.g(); ar.h();
	}

	{
		C* cp = new C;
		A* ap = cp;
		ap->f(); ap->g(); ap->h();
		delete ap;
	}

	{
		C* cp = new C;
		B* bp = cp;
        bp->f(); bp->g();
		delete bp;
	}

	return 0;
}
```

<br />

Какая функция будет вызвана?

```c++
class A {
public:
    A() {
        init();
    }

    ~A() {
        deinit();
    }

    virtual void init();
    virtual void deinit();
};

class B : public A {
public:
    B() {}
    ~B() {}

    void init() override { ... }
    void deinit() override { ... }
};

int main() {
    B b;
    return 0;
}
```

<br />

##### Виртуальные деструкторы: зачем и как

Мы уже познакомились с механизмом виртуальных функций и поняли зачем и как они вызываются. Насчёт деструкторов рассмотрим такой пример:

```c++
class Animal
{
    int age;
    
public:
    virtual void cry() = 0;
};

class Dog : public Animal
{
    std::string name;
    
public:
    void cry() override;
};

class Human : public Animal
{
    std::string name;
    std::string surname;
    
public:
    void cry() override;
};


void func()
{
    Animal* animal = nullptr;
    if (std::rand() % 2)
        animal = new Dog;
    else
        animal = new Human;    
    ...;    
    delete animal;  // что здесь делать компилятору?
}
```



Поэтому реализуем иерархии так:

```c++
class Animal
{
    int age;

public:
    virtual ~Animal() = default;  // такой деструктор - ВИРТУАЛЬНЫЙ метод
    virtual void cry() = 0;
};

// далее как обычно
```

<br />

##### const-методы

Упражнение: какая функция будет вызвана

```c++
class A
{
public:
	void f() { std::cout << "f();"; }
	void f() const { std::cout << "f() const;" }
};


int main()
{
	A a;
	a.f();

	const A ca;
	ca.f();

	const A& car = a;
	car.f();

	A& ar = a;
	ar.f();

	A* const acp = &a;
	acp->f();

	const A* cap = &a;
	cap->f();

	const A a2 = a;
	a2.f();

	return 0;
}
```

<br />

##### class vs struct

* `class` - если есть инвариант
* `struct` - если поля независимы

```c++
struct Point
{
	double x;
	double y;
};

struct Color
{
	char r;
	char g;
	char b;
};

// ???
struct Size
{
	double w;
	double h;
};

struct Rectangle
{
	Point origin;
	Size size;
};

class JuiceBottle
{
private:
	double max_volume_;
	double cur_volume_;
};

class String
{
private:
	char *s_;
	size_t len_;
	size_t capacity_;
};
```

<br />

##### Принципы дизайна класса

* RAII
* Есть инвариант - class, поля независимы - struct
* Если есть подклассы - делаем `virtual` деструктор
* Правило 6
    * `=default` где достаточно автогенерённого
    * `=delete` где нужно запретить
    
    ```c++
    class Animal
    {
    private:
        std::string name;

    public:
        Animal() : name("unknown") {}
        Animal(const Animal&) = default;
        Animal(Animal&&) noexcept = delete;
        Animal& operator= (const Animal&) = default;
        Animal& operator= (Animal&&) = delete;
        ~Animal() noexcept = default;
    };
    ```

* `noexcept` d-tor, move operators (подробнее на лекции про исключения)
* `const` на все методы, которые не меняют состояния класса (значения полей)

    ```c++
    class Animal
    {
    private:
        std::string name;

    public:
        const std::string& get_name() const noexcept { return name; }
    };
    ```

* `override` на переопределённые `virtual` методы

    ```c++
    class Animal
    {
    public:
        virtual void say() = 0;
    };

    class Cat : public Animal
    {
    public:
        void say() override;
    };
    ```

* Какой функционал делать методом, а какой - свободной фунцкией? Всопним ADL. Благодаря ему есть философский принцип, что функции, принимающие первым аргументом объект класса (или ссылку), являются частью интерфейса класса:

    ```c++
    struct Vector3D
    {
        double x, y, z;
    };

    // философски часть интерфейса класса Vector3D
    double length(const Vector3D v);
    ```

    Поэтому принцип рационального минмимума в методах:
      * struct Vector3D - всё в свободные функции (normalize, length, rotate ...))
      * vector<int> (pointer, size, capacity: size() - member; at() - member; sort() - function; empty() - ???; middle element - function; max() - function, mean() - function)
      * cache (add, get, del; clear? - в метод, т.к. эффективнее реализовать, обладая доступом до полей)

    Нужна __причина__, чтобы что-нибудь сделать методом:
        * требуется доступ до приватных полей и нельзя реализовать иначе (vector.size(), vector.at(), vector.push_back())
        * реализация через приватные поля проще и быстрее (cache.clear())
        * лучше вписывается в сщуествующие методы класса (vector.empty())
        * что-нибудь ещё?

* Никогда не используйте `protected`-поля

* Иногда нужно менять поля в `const`-методах, и это не влияет на состояние класса в умозрительном смысле. Используйте `mutable`:
    * Кеширование: бинарное дерево, которое запоминает последний запрос
    * `mutex`
    * отложенная инициализация

* SRP - Single Responsibility Principle - Принцип единственной ответственности - класс должен быть экспертом в одной и только одной области. Такие классы проще отлаживать и тестировать. Всегда понятно, ошибка в рамках его области ответственности или нет.

<br />

**Задания для закрепления материала (без баллов):**

1. Напишите структуру `Color`, задающую цвет RGB. Убедитесь, что она занимает в памяти 3 байта: `sizeof(Color) == 3`.

   Для `Color` определите операторы сравнения (`operator ==`, `operator !=`, `operator <`, `operator <=` ...).

   Для `Color` определите операторы сложения цветов `operator +` (пусть при выходе за границу в 255 выполняется арифметика по модулю 2^8) и умножения цвета на скаляр `operator *`. Напомню, что на скаляр можно умножать как слева, так и справа.

   Определите оператор вывода цвета в поток `operator <<`. Попробуйте распечатать цвет. Исправьте ошибку.

   Поиграйтесь с новой структурой, протестируйте созданные операторы.


2. Создайте иерархию классов: `Animal`, `Cat`, `Dog`. Причём `Animal` выглядит так:

   ```
   class Animal {
   public:
     Animal(const std::string& in_name) : name(in_name)
     {}
     
     virtual ~Animal() = default;
     
     virtual void say() = 0;
   
   private:
     std::string name_;
   };
   ```
   
   Напишите функцию:
   `void chorus(const std::vector<Animal *>& animals)`
   которая для каждого животного выведет его имя и звук. Например:
   
   ```
   Bobik: auh!
   Sharik: auh!
   Murzik: meow.
   ```